# Machine Learning Pipeline

Pada proyek ini akan digunakan sebuah tool untuk membuat end-to-end machine learning pipeline bernama [TFX (TensorFlow Extended)](https://www.tensorflow.org/tfx), sedangkan untuk menjalankan pipeline orchestration digunakan [Apache Beam](https://beam.apache.org/). TFX memiliki beberapa library dan komponen (TFXComponents) yang dapat digunakan untuk membangun machine learning pipeline. Library atau komponen tersebut memiliki tugas spesifik disetiap tahapannya yang mencangkup:

1. Tahap Pengolahan Data. Tahap ini terdiri dari:
    - Data Ingestion. Komponen TFX yang digunakan adalah ExampleGen, yang bertanggung jawab untuk mengumpulkan dan mengimpor data latihan dan pengujian ke dalam format yang sesuai standar.
    - Data Validation. Komponen yang digunakan adalah StatisticGen (untuk analisis statistik terhadap data), SchemaGen (untuk menentukan skema dataset, termasuk tipe data dan batasan lainnya untuk setiap fitur), dan ExampleValidator (untuk melakukan validasi lebih lanjut terhadap dataset dan mendeteksi anomali atau ketidaksesuaian dalam data).
    - Data Preprocessing. Komponen yang digunakan adalah Transform, yang bertanggung jawab untuk melakukan pembersihan dataset dan ekstraksi fitur  agar nantinya bisa digunakan dalam tahap selanjutnya.
2. Tahap Pengembangan dan Validasi Model. Tahap ini terdiri dari:
    - Proses Pengembangan Model. Komponen yang digunakan adalah Komponen Trainer (untuk melakukan proses training) dan Komponen Tuner (untuk menjalankan proses hyperparameter tuning).
    - Proses Analisis dan Validasi Model. Komponen yang digunakan adalah Komponen Resolver (untuk membandingkan model baru dengan versi model sebelumnya) dan Evaluator (untuk menjalankan proses evaluasi model).
3. Tahap Model Deployment dan Model Serving. Komponen yang digunakan adalah komponen Pusher (untuk memindahkan model yang telah dilatih dan divalidasi ke lingkungan produksi). Sedangkan untuk tipe model serving yang digunakan pada proyek ini adalah menggunakan container dengan membuat docker image dan TensorFlow Serving (TF-Serving). Model tersebut nantinya akan dijalankan menggunakan platform cloud bernama [Railway](https://railway.app/) agar bisa diakses melalui HTTP Request dengan menjalankan semua proses pipeline (pipeline orchestrator) menggunakan Apache Beam.
4. Tahap Monitoring Model Machine Learning. Pada tahap ini digunakan sebuah tool monitoring (pemantauan) dan alerting (peringatan) bernama [Prometheus](https://prometheus.io/). Prometheus adalah tool yang bersifat open source dan mampu mengumpulkan serta menyimpan metrik dalam bentuk data time series. Proses monitoring model juga akan dihubungkan dengan sebuah dashboard interaktif menggunakan bantuan tool bernama [Grafana](https://grafana.com/).

# Tahap Persiapan

## Import Dependencies

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

In [2]:
PIPELINE_NAME = "juwono136-pipeline"
SCHEMA_PIPELINE_NAME = "juwono136-tfdv-schema"
DATA_ROOT = "data"
OUTPUT_BASE = "outputs"

# create directory to store the artifacts that will be generated
PIPELINE_ROOT = os.path.join(OUTPUT_BASE, PIPELINE_NAME)

# path to a SQLite DB file to use as an MLMD (ML Metadata) storage
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'metadata.sqlite')

# output directory where created models from the pipeline will be exported
SERVING_MODEL_DIR = os.path.join(OUTPUT_BASE, 'serving_model')

In [3]:
# using InteractiveContext to store pipeline history in a folder
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Tahap Pengolahan Data

Tahap pengolahan data terdiri dari:
1. **Data Ingestion**, merupakan tahap awal dari proses pengolahan data pada machine learning pipeline pada proyek ini. Pada tahap ini, data dimasukkan dan diproses ke dalam format tertentu agar sesuai dengan standar dan dapat diakses oleh alur kerja machine learning. Pada tahap ini juga data dibagi menjadi training data dan testing data (data validasi) sehingga menghasilkan dua set data yang independen untuk mengukur kinerja model.
2. **Data Validation**. Pada tahap ini dilakukan proses validasi terhadap data yang baru dan memberi peringatan apabila terdapat anomali pada data tersebut. Proses ini dilakukan untuk memastikan parameter statistik dari data tersebut sesuai standar yang ada dan memastikan bahwa data yang digunakan untuk melatih dan menguji model memiliki kualitas yang baik dan dapat diandalkan.
3. **Data Preprocessing**. Pada tahap ini, training data akan diproses sebelum digunakan untuk melatih model machine learning. Dengan mempersiapkan data dengan baik, model diharapkan dapat belajar dari data dengan lebih efektif dan memberikan hasil yang lebih akurat.

## Data Ingestion

Pada tahap ini akan dilakukan data ingestion menggunakan komponen CsvExampleGen() yang disediakan oleh TFX. Komponen ini dipilih karena dataset yang digunakan pada proyek ini adalah format CSV.

In [4]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

Pada kode diatas, dataset dibagi dengan tujuan untuk proses training (train) dan evaluasi (eval). Pembagian ini dilakukan dengan mengatur parameter output_config dengan rasio 8:2. Rasio tersebut diatur berdasarkan jumlah dataset sebesar 7.613 sampel. Pembagian rasio tersebut menggunakan parameter hash_buckets.

example_gen_pb2 membuat objek 'Output' dari protobuffer yang digunakan untuk mengonfigurasi output dari 'CsvExampleGen'. Ini termasuk pengaturan untuk pembagian data menjadi bagian data training (train) dan evaluasi (eval). CsvExampleGen merupakan komponen TFX untuk menghasilkan data dalam format TFRecord dari file CSV. Komponen ini bertanggung jawab untuk membaca data dari sumber data, mengoversinya menjadi format yang dapat digunakan untuk TensorFlow, dan menyimpannya ke dalam TFRecord.

Untuk melihat komponen ExampleGen secara interaktif, bisa dengan menjalankan komponen tersebut menggunakan object InteractiveContext() yang sebelumnya sudah didefinisikan seperti kode dibawah ini.

In [5]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

Setelah tahap data ingestion berhasil, tahap selanjutnya adalah melakukan data validation. Pada tahap data validation, proses dibagi menjadi tahapan yang menggunakan beberapa komponen TFX berbeda yaitu membuat summary statistic dengan komponen StatisticsGen(), membuat data schema dengan komponen SchemaGen(), dan mengidentifikasi anomali pada dataset dengan komponen ExampleValidator().

### Membuat Summary Statistics

Pada tahap ini dibuat sebuah summary statistic menggunakan komponen StatisticsGen(). Komponen ini bertanggung jawab untuk menghasilkan statistik deskriptif dari dataset. Komponen ini juga akan memberikan wawasan tentang distribusi dan karakteristik fitur - fitur yang ada di dalam dataset. Komponen StatisticsGen() memiliki parameter input berupa examples untuk menerima dataset dari komponen ExampleGen.

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)

Kode di atas akan menghasilkan komponen StatisticsGen dengan detail sebagai berikut yang ditampilkan menggunakan InteractiveContext:

In [8]:
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

Untuk melihat tampilan visual dari summary statistic yang telah dibuat bisa menggunakan interactive_context.show()

In [9]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Membuat Data Schema

Setelah membuat summary statistic, langkah selanjutnya adalah membuat data schema menggunakan komponen SchemaGen(), komponen ini menerima input berupa summary statistic. Komponen SchemaGen() akan mendefinisikan tipe dan karakteristik dari setiap fitur dalam dataset, termasuk batasan dan persyaratan yang dikenakan pada nilai - nilai fitur.

In [10]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

Kode diatas akan menghasilkan komponen SchemaGen dengan detail sebagai berikut:

In [11]:
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

Untuk menampilkan data schema yang telah dibuat bisa menggunakan kode berikut ini:

In [12]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'id',INT,required,,-
'keyword',BYTES,required,,-
'location',BYTES,required,,-
'target',INT,required,,-
'text',BYTES,required,,-


Berdasarkan tampilan data schema di atas, diketahui bahwa dataset yang digunakan pada proyek ini terdiri dari 5 fitur yaitu id, keyword, location, target dan text. Fitur id dan text memiliki tipe integer, sedangkan fitur keyword, location, dan text memiliki tipe bytes. Selain itu, berdasarkan data schema di atas diketahui semua fitur tersebut haruslah lengkap (required) untuk setiap dataset.

### Mengidentifikasi Anomali Pada Dataset

Tahap selanjutnya adalah mengidentifikasi anomali pada dataset. Proses ini dilakukan menggunakan komponen ExampleValidator() yang disediakan oleh TFX. Komponen ini membutuhkan keluaran dari statistics_gen dan schema_gen sebagai parameter input. Komponen ExampleValidator() akan melakukan validasi contoh - contoh (instance) dalam dataset berdasarkan skema dan statistik yang dihasilkan serta memastikan data yang digunakan untuk melatih model memenuhi persyaratan yang telah ditentukan dalam skema.

In [14]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)

Untuk menampilkan detail dari komponen ExampleValidator yang dihasilkan bisa menggunakan kode berikut:

In [15]:
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

Untuk menampilkan hasil dari validasi komponen tersebut, bisa dengan menjalankan kode berikut:

In [16]:
interactive_context.show(example_validator.outputs["anomalies"])

Berdasarkan hasil dari validasi di atas, ternyata tidak terdapat anomali pada dataset yang digunakan pada proyek ini.

## Data Preprocessing

Setelah melakukan data validation, tahap berikutnya adalah melakukan data preprocessing. Tujuan dari tahap ini adalah mengubah data mentah menjadi data yang siap digunakan untuk melatih model. Komponen yang akan digunakan pada tahap ini adalah komponen Transform, yang akan menerima inputan berupa module file tersendiri yang berisi preprocessing function. Oleh karena itu, perlu dibuatkan terlebih dahulu sebuah module file untuk menempatkan preprocessing function tersebut.

Sebelum membuat module, terlebih dahulu didefinisikan nama dari module tersebut.

In [17]:
MODULE_BASE = "modules"
TRANSFORM_MODULE_FILE = os.path.join(MODULE_BASE, "tweet_transform.py")

Selanjutnya dibuatkan fungsi bernama transformed_name untuk mengubah nama fitur yang telah di-transform. Perubahan ini dilakukan dengan menambahkan "_xf"pada nama fitur tersebut.

In [18]:
FEATURE_KEY = 'text'
LABEL_KEY = 'target'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

Kemudian dibuatkan preprocessing function untuk melakukan proses data cleaning agar membuat data siap untuk digunakan oleh model machine learning. Karena sistem machine learning yang dibuat pada proyek ini harus mampu mengidentifikasi sebuah kalimat tweet apakah berisi bencana atau bukan bencana, maka fitur yang diambil hanya fitur 'text' saja sebagai fitur utama dan 'target' sebagai label.

In [19]:
def preprocessing_fn(inputs):
    outputs = {}

    # lowercase transformation using TensorFlow
    text = tf.strings.lower(inputs[FEATURE_KEY])

    # removing url using TensorFlow
    text = tf.strings.regex_replace(text, r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '')

    # removing non-alphabetic characters using TensorFlow
    text = tf.strings.regex_replace(text, r'[^a-z]', ' ')

    # stripping leading and trailing spaces
    text = tf.strings.strip(text)

    outputs[transformed_name(FEATURE_KEY)] = text

    # Casting label to integer
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

Pada preprocessing function diatas dilakukan proses data cleaning atau pembersihan pada teks. Terdapat beberapa tahapan proses data cleaning pada fungsi tersebut yaitu:
1. Mengambil nilai fitur yaitu 'FEATURE_KEY' dalam input, kemudian mengonversinya menjadi huruf kecil.
2. Menggunakan library re atau ekspresi reguler untuk menghapus link url karena tidak memiliki makna khusus atau tidak relevan sebagai data yang digunakan pada model.
3. Menghapus karakter selain huruf a hingga z dengan library ekspresi reguler. Ini membantu membersihkan teks dari karakter yang tidak diinginkan.
4. Menggunakan fungsi tf.strings.strip() untuk menghapus spasi di awal dan akhir teks.
5. Kemudian, menyimpan variabel 'text' yang sudah diproses ke dalam variabel outputs melalui fungsi transformed_name dari 'FEATURE_KEY'.
6. Label dari 'LABEL_KEY' dilakukan casting ke tipe data integer dan disimpan ke dalam variabel outputs. Hal ini untuk memastikan bahwa data pada fitur 'target' memiliki format integer (int64).

Untuk melihat hasil dari proses data cleaning diatas, masukkan parameter inputs yang dibutuhkan yaitu FEATURE_KEY dan LABEL_KEY ke dalam preprocessing_fn(). Jalankan kode berikut untuk melihat hasilnya:

In [20]:
inputs = {
    FEATURE_KEY: "Man wife get six years jail for setting ablaze niece http://t.co/eV1ahOUCZA",
    LABEL_KEY: 1
}

preprocessing_fn(inputs)

{'text_xf': <tf.Tensor: shape=(), dtype=string, numpy=b'man wife get six years jail for setting ablaze niece'>,
 'target_xf': <tf.Tensor: shape=(), dtype=int64, numpy=1>}

Setelah fungsi transformed_name dan preprocessing_fn berhasil dibuat dan dijalankan. Selanjutnya adalah menyimpan kedua fungsi tersebut ke dalam module file dengan menggunakan magic command %%writefile {module_name.py}. Perintah tersebut pada dasarnya akan membuatkan file dan mengisi file tersebut dengan statement yang ada di bawahnya, apabila file tersebut telah ada, perintah tersebut akan melakukan overwriting terhadap file tersebut. Module file yang dibuat ini nantinya akan digunakan pada komponen Transform.

In [21]:
%%writefile {TRANSFORM_MODULE_FILE}
"""
Transform module
"""

import tensorflow as tf
FEATURE_KEY = 'text'
LABEL_KEY = 'target'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

    
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features

    Args:
        inputs (dict): A dictionary containing input features.

    Returns:
        dict: A dictionary containing preprocessed features to transformed features.
    """

    outputs = {}

    # lowercase transformation using TensorFlow
    text = tf.strings.lower(inputs[FEATURE_KEY])

    # removing url using TensorFlow
    text = tf.strings.regex_replace(text, '((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '')

    # removing non-alphabetic characters using TensorFlow
    text = tf.strings.regex_replace(text, '[^a-z]', ' ')

    # stripping leading and trailing spaces
    text = tf.strings.strip(text)

    outputs[transformed_name(FEATURE_KEY)] = text

    # Casting label to integer
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

Overwriting modules\tweet_transform.py


Tahap berikutnya adalah mendefinisikan komponen Transform(). Komponen ini akan menerima input berupa examples untuk menerima dataset dari komponen ExampleGen pada tahap sebelumnya, schema untuk menerima data schema dari SchemaGen, dan module_file untuk menerima module file yang berisi preprocessing function.

In [23]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

Untuk melihat tampilan detail komponen Transform yang dihasilkan, jalankan kode berikut:

In [24]:
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: outputs\juwono136-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\fcc707943f924e8e945c573503c69a8c\assets


INFO:tensorflow:Assets written to: outputs\juwono136-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\fcc707943f924e8e945c573503c69a8c\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

Berdasarkan output di atas, dapat dilihat bahwa komponen Transform akan menghasilkan beberapa output mulai dari transform_graph, transform_examples (transformed data), dan sebagainya. Keseluruhan output nantinya digunakan untuk melakukan preprocessing terhadap serving set. Proses ini juga akan membantu untuk memastikan preprocessing yang dilakukan pada fase training sama dengan fase deployment sehingga mencegah feature skew.

# Tahap Pengembangan dan Validasi Model

## Mengembangkan Model

Pada tahap ini, akan dilakukan proses pembuatan dan melatih model menggunakan komponen Trainer. Sedangkan untuk proses menjalankan hyperparameter tuning atau mencari parameter model yang optimal akan menggunakan komponen Tuner.

### Komponen Trainer

Komponen Trainer membutuhkan beberapa inputan yang salah satunya adalah module file yang berisi training function. Oleh karena itu, terlebih dahulu dibuatkan sebuah module file yang berisi beberapa fungsi untuk melakukan pelatihan model.

Sebelum membuat module, terlebih dahulu didefinisikan nama dari module tersebut.

In [25]:
MODULE_BASE = "modules"
TRAINER_MODULE_FILE = os.path.join(MODULE_BASE, 'tweet_trainer.py')

Selanjutnya, dibuatkan fungsi transformed_name() untuk mengubah nama fitur yang telah melalui proses transform.

In [26]:
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

Membuat fungsi gzip_reader_fn() yang nantinya digunakan untuk memuat data yang dikompresi dalam format TFRecord dengan jenis kompresi GZIP.

In [27]:
def gzip_reader_fn(filenames):
    """Load comporessed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

Fungsi input_fn() untuk memuat transformed_feature yang dihasilkan oleh komponen Transform dan membaginya ke dalam beberapa batch. Fungsi ini terdiri dari pola file, output transformasi, jumlah epoch dan ukuran batch sebagai argumen.

In [28]:
def input_fn(file_pattern, tf_transform_output, num_epochs=None, batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # get post_transform feature spec
    transform_feature_spec = tf_transform_output.transform_feature_spec().copy()

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    
    return dataset

Kemudian, dibuatkan sebuah fungsi bernama model_builder() yang bertanggung jawab dalam membuat arsitektur model. Pada proyek ini, akan digunakan TextVectorization layer untuk mengonversi teks menjadi vektor numerik, dan Embedding layer untuk membangun arsitektur model berdasarkan vektor embedding yang dipelajari.

In [29]:
import tensorflow as tf
import tensorflow_transform as tft
from keras import layers
from keras.utils.vis_utils import plot_model

VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

def model_builder(hp):
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_embedding_dim = hp.Int('embedding_dim', min_value=8, max_value=128)

    """build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])

    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp_embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    # print model
    model.summary()
    return model

Untuk melihat arsitektur model yang dibuat. Jalankan kode berikut:

In [30]:
import keras_tuner
hp = keras_tuner.HyperParameters()

model_builder(hp)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 8)            80000     
                                                                 
 global_average_pooling1d (G  (None, 8)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 64)                576   

Setelah fungsi untuk melatih model sudah dibuat, selanjutnya adalah membuat fungsi bernama _get_serve_tf_example_fn() untuk menjalankan tahapan preprocessing data pada raw request data. Fungsi ini akan bertanggung jawab untuk melayani contoh input setelah dilakukan transformasi dan menyajikan model yang telah dilatih ke endpoint yang dapat menerima dan merespons input dari raw request data.


In [31]:
def _get_serve_tf_examples_fn(model, tf_transform_output):
    # retrieves the feature transformation layer from the TensorFlow transform output and stores it in the model
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    # defines the TensorFlow serving function to receive serialized input examples and return the model predictions
    def serve_tf_examples_fn(serialized_tf_examples):

        # retrieves the raw feature specifications from the TensorFlow transform output
        feature_spec = tf_transform_output.raw_feature_spec()

        # removes the label key from the features specifications
        feature_spec.pop(LABEL_KEY)

        # parse the serialized examples into a format usable by the model during inference
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        # applies the same transformations used during training (tft_layer) to the parsed input examples
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
    
    return serve_tf_examples_fn

Langkah berikutnya adalah membuat fungsi bernama run_fn() yang bertanggung jawab untuk menjalankan proses training model sesuai dengan parameter training yang diberikan.

In [32]:
from tfx.components.trainer.fn_args_utils import FnArgs

def run_fn(fn_args: FnArgs) -> None:
    # defines the directory for TensorBoard logs
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    # creates a TensorBoard callback to visualize the training progress
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq='batch'
    )
    # defines an Early Stopping (es) and Model Checkpoint (mc) callback that monitors the validation binary accuracy
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)

    # load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )
    # vectorize_layer.adapt(train_set.map(lambda x, y: x[transformed_name(FEATURE_KEY)]))

    # build the model
    hp = keras_tuner.HyperParameters()
    model = model_builder(hp)

    # train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000,
            validation_steps = 1000,
            epochs = 10)
    
    # defines the serving signatures for the saved model
    signatures = {
        'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'
            )
        )
    }

    # save the model
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

    # save plot model
    plot_model(
        model,
        to_file='images/model_plot.png',
        show_shapes=True,
        show_layer_names=True
    )

Berikut adalah penjelasan dari beberapa variabel yang ada di dalam fungsi run_fn() diatas:
- Variabel `log_dir` berfungsi untuk menentukan direktori untuk menyimpan log TensorBoard. Log akan disimpan dalam subdirektori bernama 'logs' di dalam direktori tempat model yang akan disajikan akan disimpan.
- Variabel `tensorboard_callback` untuk memvisualisasikan proses pelatihan. Callback ini akan mencatat log di setiap batch.
- Variabel `es` dan `mc` untuk membuat callback Early Stopping (es) dan Monitor Checkpoint (mc). Mereka berfungsi untuk menyimpan model terbaik selama pelatihan dan memonitor akurasi biner selama pelatihan.
- Variabel `tf_transform_output` untuk memuat output transformasi dari path grafik transformasi (transform_graph) dan berfungsi untuk mentransformasi fitur input selama pelatihan dan inferensi.

Seluruh fungsi diatas, kemudian disatukan ke dalam sebuah module file yang nantinya digunakan pada komponen Trainer.

In [34]:
%%writefile {TRAINER_MODULE_FILE}
"""
Trainer module
"""

import os
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner
from keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from keras.utils.vis_utils import plot_model

LABEL_KEY = 'target'
FEATURE_KEY = 'text'

def transformed_name(key):
    """Rename transformed features.

    Args:
        key (str): The original feature key.

    Returns:
        str: The transformed feature key.
    """

    return key + "_xf"

def gzip_reader_fn(filenames):
    """Load compressed data.

    Args:
        filenames (list): List of file names.

    Returns:
        tf.data.TFRecordDataset: Dataset containing compressed data.
    """

    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs=None, batch_size=64)->tf.data.Dataset:
    """Get post_transform feature & create batches of data.

    Args:
        file_pattern (str): File pattern to match input files.
        tf_transform_output (tft.TFTransformOutput): Output of the TensorFlow Transform component.
        num_epochs (int, optional): Number of epochs for dataset iteration. Defaults to None.
        batch_size (int, optional): Batch size for the dataset. Defaults to 64.

    Returns:
        tf.data.Dataset: Batched and transformed dataset.
    """
    
    # get post_transform feature spec
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    
    return dataset

VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build machine learning model.

    Args:
        hp (keras_tuner.HyperParameters): Hyperparameters for model tuning.

    Returns:
        tf.keras.Model: Compiled Keras model.
    """

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_embedding_dim = hp.Int('embedding_dim', min_value=8, max_value=128)

    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])

    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp_embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    # print model
    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Retrieve the feature transformation layer and define the serving function.

    Args:
        model (tf.keras.Model): Trained Keras model.
        tf_transform_output (tft.TFTransformOutput): Output of the TensorFlow Transform component.

    Returns:
        function: TensorFlow serving function.
    """

    # retrieves the feature transformation layer from the TensorFlow transform output and stores it in the model
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    # defines the TensorFlow serving function to receive serialized input examples and return the model predictions
    def serve_tf_examples_fn(serialized_tf_examples):

        # retrieves the raw feature specifications from the TensorFlow transform output
        feature_spec = tf_transform_output.raw_feature_spec()

        # removes the label key from the features specifications
        feature_spec.pop(LABEL_KEY)

        # parse the serialized examples into a format usable by the model during inference
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        # applies the same transformations used during training (tft_layer) to the parsed input examples
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
    
    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:
    """Define the training and serving function.

    Args:
        fn_args (FnArgs): Arguments passed to the function.
    """

    # defines the directory for TensorBoard logs
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    # creates a TensorBoard callback to visualize the training progress
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq='batch'
    )
    # defines an Early Stopping (es) and Model Checkpoint (mc) callback that monitors the validation binary accuracy
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)

    # load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )
    # vectorize_layer.adapt(train_set.map(lambda x, y: x[transformed_name(FEATURE_KEY)]))

    # build the model
    hp = keras_tuner.HyperParameters()
    model = model_builder(hp)

    # train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000,
            validation_steps = 1000,
            epochs = 10)
    
    # defines the serving signatures for the saved model
    signatures = {
        'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'
            )
        )
    }

    # save the model
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

    # save plot model
    plot_model(
        model,
        to_file='images/model_plot.png',
        show_shapes=True,
        show_layer_names=True
    )

Overwriting modules\tweet_trainer.py


Kode diatas akan menghasilkan sebuah module file dengan nama tweet_trainer.py. Module tersebut nantinya akan digunakan sebagai salah satu parameter input yang akan digunakan pada komponen Trainer.

Komponen Trainer() menerima beberapa input untuk melakukan proses pelatihan model yaitu:
- `module_file` untuk menerima module file yang mengandung training function beserta beberapa helper function.
- `examples` untuk menerima dataset dari komponen ExampleGen.
- `schema` untuk menerima data schema dari komponen SchemaGen.
- `transform_graph` untuk menerima transform graph yang dihasilkan dari komponen Transform.
- `train_args` untuk menampung parameter training.
- `eval_args` untuk menampung parameter testing atau evaluation.

In [35]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

Untuk menjalankan proses training model dan melihat tampilan detail komponen Trainer, jalankan kode berikut:

In [36]:
interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization_1 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 8)            80000     
                                                                 
 global_average_pooling1d_1   (None, 8)                0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 64)                576 


Epoch 1: val_binary_accuracy improved from -inf to 0.76524, saving model to outputs\juwono136-pipeline\Trainer\model\6\Format-Serving
INFO:tensorflow:Assets written to: outputs\juwono136-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\juwono136-pipeline\Trainer\model\6\Format-Serving\assets


1000/1000 [==============================] - 23s 21ms/step - loss: 0.3164 - binary_accuracy: 0.8609 - val_loss: 0.6861 - val_binary_accuracy: 0.7652
INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: outputs\juwono136-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\juwono136-pipeline\Trainer\model\6\Format-Serving\assets


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Komponen Tuner

Pada tahap ini digunakan komponen Tuner untuk proses mencari kombinasi optimal dari hyperparameter model dengan tujuan meningkatkan kinerja dan generalisasi model atau biasa disebut sebagai proses hyperparameter tuning. Sama seperti komponen Trainer, Komponen Tuner membutuhkan sebuah module file yang berisi tuner function sebagai salah satu input/masukkan, sehingga perlu dibuat terlebih dahulu fungsi tersebut.

Sebelum membuat module, terlebih dahulu didefinisikan nama dari module tersebut.

In [37]:
MODULE_BASE = "modules"
TUNER_MODULE_FILE = os.path.join(MODULE_BASE, 'tweet_tuner.py')

Selanjutnya, dibuatkan sebuah fungsi bernama tuner_fn() yang menerima sebuah argumen fn_args. Argumen ini berisi segala hal yang dibutuhkan untuk menjalankan proses hyperparameter tuning. Sama halnya dengan fungsi run_fn(), fungsi ini juga membutuhkan helper function, seperti input_fn() dan model_builder() yang diambil dari module file `tweet_trainer.py`.


In [38]:
%%writefile {TUNER_MODULE_FILE}
"""
Tuner module
"""

from typing import NamedTuple, Dict, Text, Any
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from keras_tuner.engine import base_tuner
from tfx.components.trainer.fn_args_utils import FnArgs

from tweet_trainer import model_builder, input_fn

# current TrainerFnArgs will be renamed to FnArgs as a util class.
TunerFnResult = NamedTuple('TunerFnResult',
                        [('tuner', base_tuner.BaseTuner),
                        ('fit_kwargs', Dict[Text, Any])])

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """Define hyperparameter tuning function for Keras Tuner.

    Args:
        fn_args (FnArgs): Arguments passed to the function.

    Returns:
        TunerFnResult: NamedTuple containing the Keras Tuner and fit_kwargs.
    """

    # define hyperparameter tuning strategy
    tuner = kt.RandomSearch(model_builder,
                        objective='val_binary_accuracy',
                        max_trials=5,
                        directory=fn_args.working_dir,
                        project_name='kt_random_search')

    # load the training and validation dataset that has been preprocessed
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    
    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', patience=3)],
            'x': train_set,
            'validation_data': val_set,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps,
    }
)

Overwriting modules\tweet_tuner.py


Berikut adalah penjelasan dari beberapa variabel yang ada di dalam fungsi tuner_fn() diatas:
- Variabel `TunerResult` yang merupakan named tuple dengan dua bidang yaitu 'tuner' dan 'fit_kwargs'. Named tuple ini digunakan untuk menggabungkan hasil fungsi tuning.
- Di dalam fungsi `tuner_fn()` dibuatkan tuner 'RandomSearch' dari library [keras tuner](https://keras.io/api/keras_tuner/tuners/random/). RandomSearch digunakan pada proses tuner kali ini, karena kemampuannya untuk menjelajahi ruang hyperparameter secara lebih efisien dan memiliki biaya komputasi yang lebih rendah. Dengan menentukan fungsi 'model_builder' sebagai model yang akan di-tune, tujuan hyperparameter tuning diatur untuk memaksimalkan akurasi biner validasi. Jumlah maksimum percobaan (max_trials) diatur menjadi 5, serta direktori untuk menyimpan hasil tuning juga ditentukan.
- Setelah proses hyperparameter tuning dilakukan, `TunerFnResult` akan mengembalikan tuner yang telah dibuat dan beserta argumen tambahan (`fit_kwargs`). Argumen fit ini mencangkup parameter yang diperlukan untuk melatih model, seperti callbacks, dataset pelatihan dan validasi, dan langkah per epoch.

Setelah module file sudah berhasil dibuat, langkah berikutnya adalah membuat komponen Tuner untuk proses model tuning. Komponen ini menerima beberapa input seperti berikut:
- Data schema yang diperoleh dari tahap data validation.
- Transformed data yang diperoleh dari tahap data preprocessing.
- Parameter tuning dan training.
- Sebuah module file yang berisi tuner function.

In [39]:
from tfx.proto import trainer_pb2

tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=50)
)

Untuk melihat detail dari komponen Tuner, jalankan kode berikut:

In [40]:
interactive_context.run(tuner)

Trial 5 Complete [00h 00m 13s]
val_binary_accuracy: 0.5556250214576721

Best val_binary_accuracy So Far: 0.7846875190734863
Total elapsed time: 00h 01m 06s
Results summary
Results in outputs\juwono136-pipeline\.temp\7\kt_random_search
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
learning_rate: 0.001
embedding_dim: 63
Score: 0.7846875190734863

Trial 2 summary
Hyperparameters:
learning_rate: 0.001
embedding_dim: 85
Score: 0.7837499976158142

Trial 3 summary
Hyperparameters:
learning_rate: 0.01
embedding_dim: 91
Score: 0.7728124856948853

Trial 1 summary
Hyperparameters:
learning_rate: 0.01
embedding_dim: 127
Score: 0.7699999809265137

Trial 4 summary
Hyperparameters:
learning_rate: 0.0001
embedding_dim: 83
Score: 0.5556250214576721


ExecutionResult(
    component_id: Tuner
    execution_id: 7
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

Berdasarkan hasil dari proses hyperparameter tuning dari komponen Tuner, didapat hasil terbaik untuk 'val_accuracy_binary' adalah sebesar 0.7846875190734863 pada Trial dengan id = 0 (Trial 0 Summary), dimana didapat values untuk learning_rate yaitu sebesar 0.001 dan embedding_dim sebesar 63. Nilai ini nantinya bisa digunakan untuk membuat model yang paling optimal.


## Analisis dan Validasi Model

Setelah proses pengembangan model berjalan lancar, tahap selanjutnya adalah membuat analisis dan validasi model. Untuk menjalankan tahap ini dibutuhkan dua buah komponen TFX yaitu komponen Resolver dan komponen Evaluator.

### Komponen Resolver

Untuk melakukan analisis dan validasi model, hal pertama yang perlu dipersiapkan adalah sebuah baseline model. Hal ini penting terutama ketika terdapat lebih dari satu versi model dan ingin membandingkan dua versi model yang berbeda. Untuk melakukannya, digunakan komponen bernama Resolver(). Komponen ini bertanggung jawab untuk meresolusi model yang akan digunakan dalam inferensi atau prediksi, dan membantu memilih versi yang tepat berdasarkan kriteria tertentu.

In [41]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

Untuk melihat hasil dari komponen Resolver, jalankan kode berikut:

In [42]:
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Komponen Evaluator

Setelah mendefinisikan Resolver, tahap selanjutnya adalah membuat beberapa konfigurasi untuk mengevaluasi model. Konfigurasi ini dibuat menggunakan library TFMA atau [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started).

In [43]:
import tensorflow_model_analysis as tfma

slicing_specs = [
    tfma.SlicingSpec(),
    tfma.SlicingSpec(feature_keys='text')
]

metrics_specs = [
    tfma.MetricsSpec(metrics=[
        tfma.MetricConfig(class_name="ExampleCount"),
        tfma.MetricConfig(class_name="AUC"),
        tfma.MetricConfig(class_name="Precision"),
        tfma.MetricConfig(class_name="Recall"),
        tfma.MetricConfig(class_name='FalsePositives'),
        tfma.MetricConfig(class_name='TruePositives'),
        tfma.MetricConfig(class_name='FalseNegatives'),
        tfma.MetricConfig(class_name='TrueNegatives'),
        tfma.MetricConfig(class_name='BinaryAccuracy', 
            threshold=tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0.5}),
                change_threshold=tfma.GenericChangeThreshold(
                    direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                    absolute={'value': 0.0001})
            )
        )

    ])
]

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='target')],
    slicing_specs=slicing_specs,
    metrics_specs=metrics_specs
)

Pada kode diatas, dilakukan pengaturan beberapa konfigurasi seperti metrik untuk mengevaluasi model beserta nilai threshold dari suatu metrik seperti 'ExampleCount' untuk jumlah contoh yang dievaluasi, 'AUC' atau Area Under the Curve, 'Precision' dan 'Recall' untuk mengukur imbalance class, 'FalsePositives', 'TruePositives', 'FalseNegatives', 'TrueNegatives' sebagai metrik untuk evaluasi biner, dan 'BinaryAccuracy' untuk akurasi biner dengan pengaturan threshold tertentu seperti terlihat pada kode diatas. Selain menentukan metrik, dilakukan juga pengaturan pembagian kelompok data berdasarkan fitur tertentu pada parameter slicing_specs.


Setelah membuat konfigurasi yang akan digunakan untuk mengevaluasi model, tahap berikutnya adalah mendefinisikan komponen Evaluator yang berfungsi untuk mengevaluasi kinerja model. Komponen ini akan menerima beberapa input seperti berikut:
- examples untuk menerima dataset dari komponen ExampleGen.
- model untuk menerima model yang dihasilkan dari komponen Trainer.
- baseline_model untuk menampung baseline model yang disediakan oleh komponen Resolver.
- eval_config untuk menerima konfigurasi untuk mengevaluasi model.

In [44]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)

Untuk melihat detail dari komponen Evaluator, jalankan kode berikut:


In [45]:
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

Hasil evaluasi dari komponen Evaluator ini dapat divisualisasikan menggunakan library TFMA dengan menjalankan kode berikut:

In [47]:
# visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)

tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

# Tahap Model Deployment dan Model Serving

Setelah model berhasil dibuat, langkah selanjutnya adalah melakukan proses deployment terhadap model. Tahap ini akan menggunakan salah satu TFXComponent bernama Pusher(). Komponen ini akan mengirim model yang telah divalidasi ke dalam target sistem deployment, baik dalam tahap training maupun proses retraining. Untuk menjalankan tugasnya, Pusher akan memeriksa apakah model yang telah dilatih telah lulus proses validasi oleh komponen Evaluator dan apakah model yang dilatih memiliki format yang sesuai dengan production environment.

Sedangkan, untuk menghubungkan machine learning pipeline yang sudah dibuat akan digunakan sebuah tools bernama TF-Serving dengan membuat dan menjalankan docker image terlebih dahulu.

### Komponen Pusher

Komponen Pusher menerima beberapa inputan berupa trained model, hasil evaluasi dari komponen Evaluator, dan argumen terkait serving file path.

In [48]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR
        )
    )
)

Untuk melihat output dari komponen Pusher, jalankan kode berikut:

In [49]:
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

### Menghubungkan Machine Learning Pipeline dengan TF-Serving

Setelah komponen Pusher dijalankan dan berhasil, akan menghasilkan sebuah folder bernama 'serving_model'. Folder inilah yang diakses oleh TF-Serving untuk mengambil model terbaru dan menjalankannya di production environment menggunakan platform cloud bernama [Railway](https://railway.app/). Proses menjalankan dan mengatur alur kerja machine learning pipeline pada proyek ini akan menggunakan [Apache Beam](https://beam.apache.org/) sebagai pipeline orchestrator melalui sebuah file bernama `local_pipeline.py`. File ini akan menyatukan dan menjalankan seluruh TFX Component (bukan lagi dengan InteractiveContext). Mulai tahap ini juga proses menjalankan sistem machine learning tidak dilakukan di jupyter notebook, tetapi akan dilakukan menggunakan teknik modular code agar sistem mudah dipelihara dan dikembangkan.

# Tahap Monitoring Model Machine Learning

Setelah model berhasil dijalankan dalam platform cloud bernama Railway dan dapat dilakukan proses HTTP Request. Langkah selanjutnya adalah melakukan monitoring model untuk memantau performa model di production environment. Pada tahap ini akan digunakan tool bernama [Prometheus](https://prometheus.io/) sebagai tool monitoring terhadap model machine learning yang dibuat. 

Pada dasarnya, TF-Serving telah menyediakan metrik endpoint yang dapat dimanfaatkan oleh tool monitoring, seperti Prometheus. Untuk melakukan hal ini, perlu dijalankan TF-Serving dan Prometheus secara bersamaan (side by side). Pada proses monitoring, Prometheus akan melakukan proses pengumpulan (pull) metrik secara kontinu melalui REST endpoint.

Pada proyek ini juga, tool prometheus akan di sinkronkan dengan Grafana agar bisa menampilkan dashboard interaktif yang menarik dalam proses monitoring. Seluruh tahapan pembuatan mekanisme monitoring model machine learning tidak dilakukan di jupyter notebook, tetapi akan dilakukan dengan kode terpisah atau secara modular code agar sistem atau kode mudah dipelihara dan dikembangkan.